In [ ]:
import os
import pandas as pd

directory = "/Users/vincewu/Bees-Project"
processed_folder = "Fully Processed"
processed_file = 'processed_orientation.csv'

processed_csv = pd.read_csv(os.path.join(directory, processed_folder, processed_file), engine='python', header=None)

headers=processed_csv.loc[0].tolist()
headers[0]='UID'

processed_csv.columns=headers
processed_csv=processed_csv.drop(labels=0, axis=0)
processed_csv.index-=1
processed_csv['Date']=pd.to_datetime(processed_csv["Date"])
processed_csv



In [ ]:
processed_csv.iloc[0].tolist()

In [ ]:
forager_dates=pd.read_csv(os.path.join(directory, processed_folder, 'forager_dates.csv'), parse_dates=['Forager Date'])
forager_dates.index = forager_dates.iloc[:, 0]
forager_dates=pd.DataFrame(forager_dates.iloc[:, 1])
forager_dates

In [ ]:
'25SQCMICSBB745E009BD3' in forager_dates


In [ ]:
forager_dates

In [ ]:
df=[]
for i in range(len(processed_csv)):
    if processed_csv['Group'][i]=='New':
        uid = processed_csv['UID'][i]
        date = processed_csv['Date'][i]
        # print(uid)
        if uid in forager_dates:
            print(forager_dates.loc[uid])

        if uid in forager_dates and date<forager_dates.loc[uid]:
            df.append(processed_csv.iloc[i].tolist())


In [ ]:
trips={}
forager_dates={}
# orientation_data=pd.DataFrame([0 for i in range(len(processed_csv.columns.tolist()))], columns=processed_csv.columns.tolist())
orientation_data=[]
for i in range(len(processed_csv)):
    if processed_csv['Group'][i]=='New':
        uid = processed_csv['UID'][i]
        date=processed_csv['Date'][i]
        #print(uid, date)
        total_duration=float(processed_csv['Trip Average Duration (Hours)'][i])*int(processed_csv['Trip Amount'][i])
        if uid not in trips:
            trips[uid] = [[date, total_duration]]
        else:
            trips[uid].append([date, total_duration])
        if uid not in forager_dates:
            if total_duration>=0.5:
                # if len(trips[uid])>=2:
                #     if trips[uid][-1][1]>=2*trips[uid][-2][1] and trips[uid][-1][1]>=0.5:
                #         # if uid == '25SQCMICSBB042700B285':
                #             # print(trips[uid][-3][1], trips[uid][-3][1])
                #             # print(trips[uid])
                #         forager_dates[uid]=trips[uid][-1][0]
                if len(trips[uid])>=3:
                    if trips[uid][-2][1]>=2*trips[uid][-3][1] and trips[uid][-2][1]>=0.5:
                        # if uid == '25SQCMICSBB042700B285':
                            # print(trips[uid][-3][1], trips[uid][-3][1])
                            # print(trips[uid])
                        forager_dates[uid]=trips[uid][-2][0]

            if not uid in forager_dates:
                # orientation_data=pd.concat([orientation_data, processed_csv.iloc[i]], axis=0, ignore_index=True)
                orientation_data.append(processed_csv.iloc[i].tolist())

# forager_dates

# sum=0
# for uid in trips:
#     for i in range(1, len(trips[uid])):
#         if trips[uid][i][1]>=2*trips[uid][i-1][1] and trips[uid][i][1]>1 and trips[uid][i-1][1]<0.5:
#             if i<len(trips[uid])-1:
#                 if trips[uid][i+1][1]<2*trips[uid][i-1][1] and uid in date_found:
#                     print(uid, trips[uid][i+1][0])
#                     sum+=1

# print(sum)

df = pd.DataFrame.from_dict(forager_dates, orient='index', columns=['Forager Date'])
# df.to_csv(directory+'/forager_dates.csv')
trips



In [ ]:
pd.DataFrame(orientation_data).to_csv(os.path.join(directory, processed_folder, 'orientation_data.csv'))

In [ ]:
uids=set()
# new_dfs=[[], []]
new_dfs=[]
for i in range(len(processed_csv)):
    if int(processed_csv['Trip Amount'][i])==0:
        continue
    uid = processed_csv['UID'][i]
    if processed_csv['Group'][i]=='Catch':
        if uid in uids:
            new_dfs.append(processed_csv.iloc[i])
        # tagged bees were released in evening
        elif processed_csv['Date'][i]==pd.to_datetime('2023-01-25'):
            uids.add(uid)
            new_dfs.append(processed_csv.iloc[i])
        else:
            uids.add(uid)
    elif processed_csv['Group'][i]=='New':
        if uid in forager_dates and processed_csv['Date'][i] >= forager_dates[uid]:
            if int(processed_csv['Trip Amount'][i]) > 0:
                new_dfs.append(processed_csv.iloc[i])

# new_dfs
# non_forager_data=pd.DataFrame(new_dfs[0])
forager_data=pd.DataFrame(new_dfs)
# non_forager_data.to_csv(os.path.join(directory, processed_folder, 'non_forager_data.csv'))
forager_data.to_csv(os.path.join(directory, processed_folder, 'forager_data.csv'))
